In [30]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard
import datetime
import pickle
print('Versão do TensorFlow: ', tf.__version__)

Versão do TensorFlow:  2.16.1


## Test GPU

In [31]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [1]:
import numpy as np

# Ações
# 0 Nada
# 1 Ir para a Esquerda 
# 2 Ir para a Direita
# 3 Girar para a Esquerda
# 4 Girar para a Direita
# 5 Ligar o Motor

# Gerar os dados conforme especificado

data_rotate_left = []
data_rotate_right = []
data_go_right = []
data_go_left = []
data_turn_on_engine = []

# Preencher as listas de dados conforme as condições especificadas
for sensor_activated in [True, False]:
    for x in range(900):
        for currentX in range(900):
            for angle in range(-180, 181):
                    if sensor_activated:
                        data_turn_on_engine.append([x, currentX, angle,1, 5])  # Ligar o Motor
                    elif -90 <= angle <= -5:
                        data_rotate_left.append([x, currentX, angle,0, 3])  # Girar para a Esquerda
                    elif 5 <= angle <= 90:
                        data_rotate_right.append([x, currentX, angle,0, 4])  # Girar para a Direita
                    elif -5 < angle < 5:
                        if currentX < x:
                            data_go_right.append([x, currentX, angle,0, 2])  # Ir para a Direita
                        elif currentX > x:
                            data_go_left.append([x, currentX, angle,0, 1])  # Ir para a Esquerda


: 

: 

: 

In [ ]:

# Determinar o tamanho mínimo entre as listas para balanceamento
min_size = min(len(data_rotate_left), len(data_rotate_right), len(data_go_right), len(data_go_left), len(data_turn_on_engine))

# Balancear as listas
balanced_data = (
    data_rotate_left[:min_size] +
    data_rotate_right[:min_size] +
    data_go_right[:min_size] +
    data_go_left[:min_size] +
    data_turn_on_engine[:min_size]
)

# Contar a quantidade de cada tipo
count_rotate_left = sum(1 for item in balanced_data if item[4] == 3)
count_rotate_right = sum(1 for item in balanced_data if item[4] == 4)
count_go_right = sum(1 for item in balanced_data if item[4] == 2)
count_go_left = sum(1 for item in balanced_data if item[4] == 1)
count_turn_on_engine = sum(1 for item in balanced_data if item[4] == 5)

# Mostrar a quantidade de cada tipo
print("Quantidade de 'RotateLeft':", count_rotate_left)
print("Quantidade de 'RotateRight':", count_rotate_right)
print("Quantidade de 'Go Right':", count_go_right)
print("Quantidade de 'Go Left':", count_go_left)
print("Quantidade de 'TurnOnEngine':", count_turn_on_engine)

# Converter para um array numpy 2D
balanced_data = np.array(balanced_data, dtype=int)

X = balanced_data[:, :4]  # Features: x, currentX e angle
y = balanced_data[:, 4]   # Labels: 1, 2, 3, 4 ou 5

print("Tamanho do balanced_data:", len(balanced_data))

In [44]:

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [45]:

# Salvar os dados em um arquivo .pkl
with open('data.pkl', 'wb') as f:
    pickle.dump([X_train, y_train, X_test, y_test], f)


In [46]:
# Carregar os dados do arquivo .pkl
with open('./data.pkl', 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)

In [36]:
%load_ext tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Construir o modelo
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(3,)),  # Ajustar a entrada para 3 features
    tf.keras.layers.Dense(64, activation='relu'),  # Ajustar a entrada para 3 features
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')  # 5 saídas: 0, 1, 2, 3, 4
])

# Compilar o modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(X_train, y_train, epochs=100, batch_size=32000, validation_split=0.2, callbacks=[tensorboard_callback])


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
292/292 ━━━━━━━━━━━━━━━━━━━━ 12s 21ms/step - accuracy: 0.8541 - loss: 2.3165 - val_accuracy: 0.9960 - val_loss: 0.0138
Epoch 2/100
292/292 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9957 - loss: 0.0122 - val_accuracy: 0.9963 - val_loss: 0.0095
Epoch 3/100
292/292 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9961 - loss: 0.0095 - val_accuracy: 0.9966 - val_loss: 0.0079
Epoch 4/100
292/292 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9967 - loss: 0.0078 - val_accuracy: 0.9978 - val_loss: 0.0062
Epoch 5/100
292/292 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9973 - loss: 0.0066 - val_accuracy: 0.9980 - val_loss: 0.0053
Epoch 6/100
292/292 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9980 - loss: 0.0054 - val_accuracy: 0.9991 - val_loss: 0.0035
Epoch 7/100
292/292 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9985 - loss: 0.0042 - val_accuracy: 0.9966 - val_loss: 0.0079
Epoch 8/100
292/292 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9982 - loss: 0.0050 - val_a

In [37]:

# Avaliar o modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Model Accuracy: {accuracy * 100:.2f}%')


91024/91024 ━━━━━━━━━━━━━━━━━━━━ 181s 2ms/step - accuracy: 0.9997 - loss: 4.9418e-04
Model Accuracy: 99.97%


In [56]:
# 0 Nothing
# 1 Go Left
# 2 Go Right
# 3 RoteteLeft
# 4 RotateRight
# 5 TurnOnEngine

# GO TO RIGHT
targetX = 680
agentX = 340
angle = 0
class_names = ['Nothing', 'Go Left', 'Go Right',
               'RoteteLeft', 'RotateRight', 'TurnOnEngine']
predictions = model.predict(np.array([[targetX, agentX, angle]]))
 
score = tf.nn.softmax(predictions[0])
print(
    "{}-Must be GO TO RIGHT: {} - with a {:.2f} percent confidence."
    .format(np.argmax(score),class_names[np.argmax(score)], 100 * np.max(score))
)

# GO TO LEFT
targetX =340  
agentX = 680
angle = 0

predictions = model.predict(np.array([[targetX, agentX, angle]]))
score = tf.nn.softmax(predictions[0])
print( "{}-Must be GO TO LEFT: {} - with a {:.2f} percent confidence."
    .format(np.argmax(score),class_names[np.argmax(score)], 100 * np.max(score))
)

# ROTATE LEFT
targetX = 680
agentX = 340
angle = -30

predictions = model.predict(np.array([[targetX, agentX, angle]]))
score = tf.nn.softmax(predictions[0])
print( "{}-Must be ROTATE LEFT: {} - with a {:.2f} percent confidence."
    .format(np.argmax(score),class_names[np.argmax(score)], 100 * np.max(score))
)

# ROTATE RIGHT
targetX = 680
agentX = 340
angle = -30

predictions = model.predict(np.array([[targetX, agentX, angle]]))
score = tf.nn.softmax(predictions[0])
print( "{}-Must be ROTATE RIGHT: {} - with a {:.2f} percent confidence."
    .format(np.argmax(score),class_names[np.argmax(score)], 100 * np.max(score))
)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2-Must be GO TO RIGHT: Go Right - with a 40.46 percent confidence.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1-Must be GO TO LEFT: Go Left - with a 40.46 percent confidence.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
2-Must be ROTATE LEFT: Go Right - with a 40.46 percent confidence.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
2-Must be ROTATE RIGHT: Go Right - with a 40.46 percent confidence.


In [39]:
%tensorboard

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2024-06-23 18:03:22.827942: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Error: A logdir or db must be specified. For example `tensorboard --logdir mylogdir` or `tensorboard --db sqlite:~/.tensorboard.db`. Run `tensorboard --helpfull` for details and examples.

In [40]:
model.save("./models/model_saved.h5")
model.save(filepath="./models/model_saved.keras")


 

#### The command below is to copy from the docker container to local folder

In [41]:
!docker cp cool_spence:/tf/models/. /mnt/d/git/victorbarbosa/src/assets/models/

/usr/bin/sh: 1: docker: not found


### From here to you only MACOS to test 

In [42]:
import tensorflow as tf
import numpy as np
model = tf.keras.models.load_model("./models/model_saved.keras")

targetX = 0
agentX = 340
angle =  30

predictions = model.predict(np.array([[targetX, agentX, angle]]))
# 0 Nothing
# 1 Go Left
# 2 Go Right
# 3 RotateLeft
# 4 RotateRight
# 5 TurnOnEngine
class_names = ['Nothing', 'Go Left', 'Go Right',
               'RotateLeft', 'RotateRight', 'TurnOnEngine']

score = tf.nn.softmax(predictions[0])
print(
    "{} This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(np.argmax(score),class_names[np.argmax(score)], 100 * np.max(score))
)

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 913ms/step
4 This image most likely belongs to RotateRight with a 40.46 percent confidence.


In [43]:
# !pip install tensorflowjs
from tensorflow.keras.models import load_model
import tensorflow as tf

# Carregar o modelo do arquivo .h5
model = load_model('./models/model_saved.h5')

# Compile o modelo (você pode precisar ajustar a função de perda, otimizador e métricas conforme necessário)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Salvar o modelo no formato SavedModel
tf.saved_model.save(model, './models/saved_model/')

!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model './models/saved_model/' models/web_model

INFO:tensorflow:Assets written to: ./models/saved_model/assets


INFO:tensorflow:Assets written to: ./models/saved_model/assets


2024-06-23 18:03:34.099253: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-23 18:03:40.223137: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-23 18:03:40.292908: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-23 18:03:40.292966: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUM